In [ ]:
import scanpy as sc
import anndata as ad
import pandas as pd
import warnings
import scanpy.external as sce

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import copy

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# ==============================================================================
# 1. Define File Information
# ==============================================================================
files_info = [
    {"path": "/public/home/design1/data/cellranger/data_CARNK_START/CAR-NK/adata_B_CAR_NK.h5ad", "group": "CAR_NK"},
    {"path": "/public/home/design1/data/cellranger/data_CARNK_START/START/adata_B_START.h5ad", "group": "START"},
    {"path": "/public/home/design1/data/cellranger/data_e_cart/h5ad_process/20250928/sc_data/adata_B.h5ad", "group": "ECART"}
]

# ==============================================================================
# 2. Loop through reading, processing, and filtering data
# ==============================================================================
adatas = []

for info in files_info:
    print(f"Reading: {info['group']} ...")
    adata = sc.read_h5ad(info['path'])
    
    # -------------------------------------------------------
    # Special handling for different datasets
    # -------------------------------------------------------
    if info['group'] == 'ECART':
        print(f"  -> ECART specific: Restoring .raw (Norm+Log) to .X")
        if adata.raw is not None:
            adata = adata.raw.to_adata()
        else:
            print("  Error: ECART data has no .raw, cannot restore full genome!")

    elif info['group'] == 'START':
        print(f"  -> START data filtering:")
        print(f"    Cell count before filtering: {adata.n_obs}")
        
        # Define samples and stages to exclude
        samples_to_exclude = ['10900304', '10900315', '10900316']
        stages_to_exclude = ['M4', 'M5', 'M12']
        
        if 'sample' not in adata.obs.columns or 'stage' not in adata.obs.columns:
            print("  Error: START data missing 'sample' or 'stage' columns!")
        else:
            adata.obs['sample'] = adata.obs['sample'].astype(str)
            # Create mask: exclude specific samples and stages
            mask = (~adata.obs['sample'].isin(samples_to_exclude)) & (~adata.obs['stage'].isin(stages_to_exclude))
            adata = adata[mask].copy()
            print(f"    Excluded samples: {samples_to_exclude}")
            print(f"    Excluded stages: {stages_to_exclude}")
            print(f"    Cell count after filtering: {adata.n_obs}")

    # -------------------------------------------------------
    # Harmonize Metadata
    # -------------------------------------------------------
    adata.obs['Sample_Group'] = info['group']
    adata.obs_names_make_unique()
    
    # Standardize 'sample' column
    if 'sample' not in adata.obs.columns:
        if 'sample_id' in adata.obs.columns:
            adata.obs['sample'] = adata.obs['sample_id'].astype(str)
    
    # Standardize 'stage' column and rename Baseline to BL
    if 'stage' in adata.obs.columns:
        adata.obs['stage'] = adata.obs['stage'].replace(
            {'Baseline': 'BL', 'baseline': 'BL', 'BaseLine': 'BL'}
        )

    adatas.append(adata)

# ==============================================================================
# 3. Concatenate (Merge)
# ==============================================================================
print("\nMerging objects...")
adata_B = ad.concat(adatas, join='outer', label='batch', keys=[f['group'] for f in files_info], index_unique='-', fill_value=0)
print(f"Merge complete: {adata_B.shape}")

In [ ]:
import time
import scanpy.external as sce

adata_B.raw = adata_B.copy()
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start highly_variable_genes')
sc.pp.highly_variable_genes(adata_B, n_top_genes=3000, batch_key='sample', subset=False)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start regress_out')
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start scale')
sc.pp.scale(adata_B, max_value=10)

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start pca')
sc.tl.pca(adata_B, svd_solver="arpack", use_highly_variable=True)

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start harmony')
sce.pp.harmony_integrate(adata_B, ['Sample_Group','sample_id'],sigma=0.6,**{'max_iter_harmony' : 30})
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start neighbors')
sc.pp.neighbors(adata_B,  use_rep='X_pca_harmony', n_neighbors=15)

print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start leiden')
sc.tl.leiden(adata_B,key_added='leiden_B')
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start louvain')
sc.tl.louvain(adata_B,key_added='louvain_B')
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start umap')
sc.tl.umap(adata_B)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),' - Start end')



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize']=(6,6)
sns.set(style="ticks", font_scale=1.5)

sc.pl.umap(adata_B, 
           color = ['Sample_Group','cell_type_2','cell_type_3'],#'leiden_B4',
           s = 8,vmin='p1',vmax='p99',
           ncols=4, cmap='Spectral_r', 
          # wspace=.3,
          legend_loc = 'on data',
           show=True,
          )

In [ ]:
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. Define Core Parameters (Specify Order) ---
# 1.1 Custom marker display order (adjust the list order directly)
marker = [
    'MS4A1','CD19','IGHD','IGHM','MME','CD24','GPR183',
    'CD27','COCH','FCRL5','ITGAX','TNFRSF1B','CD69','XBP1',
    'PRDM1','MZB1','IGHA1','IGHG1','TNFRSF17','MKI67'
]  # The dotplot will strictly follow this sequence.

# 1.2 Custom grouping order for 'cell_type_3'
# Arrange these categories in your desired biological or chronological order.
cell_type_order = [
    'Naive B', 'Immature B', 'Unsw mem B', 'Sw mem B',
    'Atypical B', 'Activated B', 'Plasma cells', 'Plasmablast'
]

# --- 2. Fix Category Order for 'cell_type_3' (Critical: Prevents Auto-sorting) ---
# Convert 'cell_type_3' to a categorical variable with the specified order.
adata_B.obs['cell_type_3'] = pd.Categorical(
    adata_B.obs['cell_type_3'],
    categories=cell_type_order,  # Sequence of priority for display
    ordered=True                 # Enable ordered classification
)

# --- 3. Generate Dotplot (Strictly Following Specified Order) ---
sns.set(style="ticks", font_scale=1)
sc.pl.dotplot(
    adata_B, 
    var_names=marker,          # Gene order = defined marker list
    groupby="cell_type_3",     # Grouping order = defined cell_type_order
    show=False,
    standard_scale='var',      # Normalize expression per gene (0 to 1)
    dendrogram=False,          # MUST be False to maintain custom grouping order
    # figsize=(24,6),          # Optional: Adjust figure size if needed
    swap_axes=False            # Set to True for cell types on X-axis and genes on Y-axis
)

plt.show()
plt.close()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize']=(6,6)
sns.set(style="ticks", font_scale=1.5)

cell_colors = {
    'Activated B': '#6756cc',    
    'Atypical B': '#4d90fe',    
    'Immature B': '#80CDC2',   
    'Naive B': '#A0D1F1',      
    'Naïve B':'#A0D1F1',  
    'Plasma cells': '#9b59b6',  
    'Sw mem B': "#07A0C3",     
    'Unsw mem B': '#FFF1D0',    
    'Plasmablast': "#ff69b4"
}

sc.pl.umap(adata_B, 
           color = ['cell_type_3'],#
           s = 8,vmin='p1',vmax='p99',
           ncols=4, cmap='Spectral_r', 
          # wspace=.3,
           legend_loc = 'on data',
           palette=cell_colors,
           save="1128_B_celltype1-name.pdf",
           show=True,
          )

In [ ]:
cell_colors = {
    'Activated B': '#6756cc',   
    'Atypical B': '#4d90fe',  
    'Immature B': '#80CDC2',   
    'Naive B': '#A0D1F1',      
    'Naïve B':'#A0D1F1',  
    'Plasma cells': '#9b59b6',  
    'Sw mem B': "#07A0C3",     
    'Unsw mem B': '#FFF1D0',   
    'Plasmablast': "#ff69b4"
}
x_min = np.min(adata_B.obsm['X_umap'][:,0])-0.5
x_max = np.max(adata_B.obsm['X_umap'][:,0])+0.5
y_min = np.min(adata_B.obsm['X_umap'][:,1])-0.5
y_max = np.max(adata_B.obsm['X_umap'][:,1])+0.5

sns.set(style="ticks", font_scale=1.5)
ncol = 8
fig, axes = plt.subplots(nrows=1, ncols=9, figsize=(26,2.5))
n=0
for i in ['BL','D0','D14','M1', 'M2', 'M3','M6','M9','M12']:
    n+=1
    if n % ncol == 0:
        c = ncol-1
        r =  n // ncol -1
    else:
        r =  n // ncol
        c = (n % ncol)-1
    ax = sc.pl.umap(adata_B[(adata_B.obs.stage==i)],#[(adata_B.obs.stage=='BL')&(adata_B.obs.sample_id=='ZXY1-10900302-BL') ],
               color = ['cell_type_3'], s = 5,
               legend_loc = None,
               wspace=-1,
               #frameon=False,
               vmax='p99.9',
               ncols=2,
               palette=cell_colors,
               title=i,
               ax=axes[n-1], show=False
              )
    ax.set_xlabel('')  
    ax.set_ylabel('') 
    ax.set_xlim(x_min,x_max)
    ax.set_ylim(y_min,y_max)
    
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.05, hspace=0.05)
save_path = '/public/home/design1/data/cellranger/data_CARNK_START/output/B_UMAP_cell_stage.pdf'
plt.savefig(
    save_path,
    dpi=300,  
    transparent=True,
    bbox_inches='tight',
)
plt.show()


In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import os

# file_path = '/public/home/design1/data/cellranger/data_CARNK_START/h5ad/adata_B_1128.h5ad'
# adata_B = sc.read(file_path)

groupby_col = 'stage'  

# 2.(Key Markers)

pathway_genes = {
    'B Cell Activation': [
        'CD69', 'CD83', 'CD86', 'MYC', 'BATF', 'TRAF6'
    ],
    'BCR Signaling': [
        'CD79A', 'CD79B', 'SYK', 'BTK', 'BLK', 'LYN', 'PLCG2', 'PIK3CD'
    ],
    'Response to IL-4': [
        'IL4R', 'STAT6', 'FCER2', 'BATF', 'IRF4'
    ],
    'B Cell Proliferation': [
        'MKI67', 'TOP2A', 'PCNA', 'MCM6', 'CDK1'
    ],
    'Ag Processing (MHC II)': [
        'HLA-DRA', 'HLA-DRB1', 'HLA-DQA1', 'HLA-DQB1', 'HLA-DPB1', 'CD74'
    ],
    'Ig Production': [
        'XBP1', 'PRDM1', 'MZB1', 'SEC61A1', 'IGJ' 
    ],
    'Humoral Response': [
        'CD40', 'CD40LG', 'CXCR5', 'AICDA', 'ICOSLG'
    ],
    'Somatic Hypermutation': [
        'AICDA', 'UNG', 'POLH', 'MSH2', 'MSH6' 
    ]
}

# 3.filter
var_names = adata_B.var_names
pathway_genes_filtered = {
    k: [g for g in v if g in var_names] 
    for k, v in pathway_genes.items()
}

print("Gene list:")
for k, v in pathway_genes_filtered.items():
    print(f"{k}: {len(v)} genes -> {v}")

# 4.  Dotplot
sc.pl.dotplot(
    adata_B,
    var_names=pathway_genes_filtered,
    groupby=groupby_col,
    standard_scale='var',  
    dendrogram=False,     
    title='Expression of Key Pathway Genes over Time',
    show=False,
    swap_axes=True       
)

#save PDF
save_dir = '/public/home/design1/data/cellranger/data_CARNK_START/output/'
os.makedirs(save_dir, exist_ok=True)  
plt.savefig(
    os.path.join(save_dir, 'pathway_dotplot.pdf'),  
    bbox_inches='tight',  
    dpi=300,  
    format='pdf'  
)


plt.show()
plt.clf()

In [ ]:
from py_monocle import learn_graph, order_cells 

cell_colors = {
    'Activated B': '#6756cc',   
    'Atypical B': '#4d90fe',    
    'Immature B': '#80CDC2',    
    'Naive B': '#A0D1F1',       
    'Naïve B':'#A0D1F1',  
    'Plasma cells': '#9b59b6', 
    'Sw mem B': "#07A0C3",     
    'Unsw mem B': '#FFF1D0',     
    'Plasmablast': "#ff69b4"
}

# adata_B = sc.read_h5ad("path_to_your_adata_B.h5ad")

# Prepare data for trajectory analysis
umap = adata_B.obsm["X_umap"].copy()
clusters = adata_B.obs["leiden_B7"].cat.codes.values
adata_B.obs["cell_type_3"] = adata_B.obs["cell_type_3"].astype("category")

#Main trajectory analysis
projected_points, mst, centroids = learn_graph(
    matrix=umap,
    clusters=clusters,
    k_nn=10,
    n_centroids=5
)

#Select root cell (starting point) based on biological knowledge (e.g., Naive B cells)
naive_b_mask = adata_B.obs["cell_type_3"] == "Immature B"
if naive_b_mask.sum() > 0:
    naive_b_indices = np.where(naive_b_mask)[0]
    root_cell = naive_b_indices[np.argmin(umap[naive_b_indices, 0])]
else:
    root_cluster = adata_B.obs["leiden_B7"].value_counts().index[0]
    root_cluster_mask = adata_B.obs["leiden_B7"] == root_cluster
    root_cell = np.where(root_cluster_mask)[0][0]

# Compute pseudotime
pseudotime = order_cells(
    umap, 
    centroids,
    mst=mst,
    projected_points=projected_points,
    root_cells=root_cell
)

#Create directory for saving figures
fig_dir = r"/public/home/design1/data/cellranger/data_CARNK_START/output/附图"
os.makedirs(fig_dir, exist_ok=True)

plt.figure(figsize=(10, 8))
plt.title("B Cell Subset Developmental Trajectory")

cell_type_colors = [cell_colors.get(ct, "#808080") for ct in adata_B.obs["cell_type_3"]]
plt.scatter(umap[:, 0], umap[:, 1], c=cell_type_colors, s=3, alpha=0.8)

edges = np.array(mst.nonzero()).T
for edge in edges:
    plt.plot(centroids[edge, 0], centroids[edge, 1], c="black", linewidth=1.5)

# MMark root cell
plt.scatter(umap[root_cell, 0], umap[root_cell, 1], 
           c="red", s=150, marker="*", label="Root Cell")

handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, 
                      markersize=8, label=ct) 
           for ct, color in cell_colors.items()]
plt.legend(handles=handles, title="Cell Type", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.show()
plt.savefig(os.path.join(fig_dir, "B_cell_trajectory_celltype.pdf"), dpi=300, bbox_inches='tight')
# plt.savefig(os.path.join(fig_dir, "B_cell_trajectory_celltype.png"), dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(10, 8))
plt.title("B Cell Pseudotime Distribution")
scatter = plt.scatter(umap[:, 0], umap[:, 1], c=pseudotime, 
                     s=3, cmap="plasma", alpha=0.8)
plt.colorbar(scatter, label="Pseudotime")

for edge in edges:
    plt.plot(centroids[edge, 0], centroids[edge, 1], c="black", linewidth=1.5)

plt.scatter(umap[root_cell, 0], umap[root_cell, 1], 
           c="red", s=150, marker="*", label="Root Cell")

plt.legend()
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, "B_cell_trajectory_pseudotime.pdf"), dpi=300, bbox_inches='tight')
plt.savefig(os.path.join(fig_dir, "B_cell_trajectory_pseudotime.png"), dpi=300, bbox_inches='tight')
plt.show()
plt.close()

print(f"Results saved to {fig_dir}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# 1. Load and Preprocess 
df = pd.read_csv("/public/home/design1/data/cellranger/data_CARNK_START/output/merged_B_cell_counts_final.csv")

# Update Stage Mapping
df['Stage_Std'] = df['Stage'].replace({'Baseline': 'BL'})

# Define the strict chronological order
stage_order = ['BL', 'D0', 'D14', 'M1', 'M2', 'M3', 'M6', 'M9', 'M12']
df['Stage_Std'] = pd.Categorical(df['Stage_Std'], categories=stage_order, ordered=True)

# Sort
df = df.sort_values(['Sample_Group', 'Sample', 'Cell_Type', 'Stage_Std'])

# Filter out M9 and M12 for PCA 
df = df[~df['Stage_Std'].isin(['M9', 'M12'])]
df['Stage_Std'] = df['Stage_Std'].cat.remove_unused_categories()

# Prepare Data for PCA (Pivot)
df_sub = df[df['Cell_Type'] != 'B cells'].copy()
pivot_df = df_sub.pivot_table(index=['Sample_Group', 'Sample', 'Stage_Std'], 
                              columns='Cell_Type', 
                              values='Count', 
                              aggfunc='mean').fillna(0)

# Normalize to frequencies
row_sums = pivot_df.sum(axis=1)
valid_rows = row_sums > 0
pivot_pct = pivot_df[valid_rows].div(row_sums[valid_rows], axis=0)

def plot_pca_refined_v3(data_df, title_suffix, output_filename):
    if data_df.empty:
        print(f"No data available for {title_suffix}")
        return

    # Run PCA
    pca = PCA(n_components=2)
    coords = pca.fit_transform(data_df)
    
    plot_df = data_df.reset_index()
    plot_df['PC1'] = coords[:, 0]
    plot_df['PC2'] = coords[:, 1]
    
    exp_var = pca.explained_variance_ratio_

    # Plot Setup
    plt.figure(figsize=(12, 10))
    ax = plt.gca()

    unique_stages = plot_df['Stage_Std'].unique().sort_values()
    n_stages = len(unique_stages)
    palette_colors = sns.color_palette('viridis', n_colors=n_stages)
    stage_color_map = dict(zip(unique_stages, palette_colors))


    centroids = plot_df.groupby('Stage_Std')[['PC1', 'PC2']].mean()
    centroids = centroids.reindex(unique_stages)
    
    plt.plot(centroids['PC1'], centroids['PC2'], 
             color='black', linestyle='--', linewidth=2.5, alpha=0.6, zorder=1, label='Trajectory')

    for i in range(len(centroids) - 1):
        p1 = centroids.iloc[i]
        p2 = centroids.iloc[i+1]
        plt.arrow(p1.PC1, p1.PC2, (p2.PC1-p1.PC1)*0.8, (p2.PC2-p1.PC2)*0.8, 
                  head_width=0.04, color='black', alpha=0.6, zorder=1)


    sns.scatterplot(data=plot_df, x='PC1', y='PC2', hue='Stage_Std', 
                    s=250, alpha=0.85, palette=stage_color_map, 
                    marker='o', edgecolor='white', linewidth=1.0, 
                    legend='full', ax=ax, zorder=2)

    target_stages = ['BL', 'D0', 'D14', 'M1', 'M2', 'M3', 'M6'] 
    target_stages = [s for s in target_stages if s in unique_stages]

    std_multiplier = 1.5 
    
    for stage in target_stages:
        stage_data = plot_df[plot_df['Stage_Std'] == stage]
        
        if len(stage_data) > 1:
            color_val = stage_color_map[stage]
            
            confidence_ellipse(stage_data['PC1'], stage_data['PC2'], ax, n_std=std_multiplier,
                               edgecolor=color_val, 
                               facecolor=color_val, 
                               alpha=0.08, 
                               linestyle='-',       
                               linewidth=3.0, 
                               zorder=1)
            
            mean_x = stage_data['PC1'].mean()
            mean_y = stage_data['PC2'].mean()
            
            ax.text(mean_x, mean_y, stage, fontsize=20, fontweight='bold', 
                    color=color_val, ha='center', va='center',
                    bbox=dict(boxstyle="round,pad=0.2", fc="white", ec=color_val, alpha=0.8, lw=2.0),
                    zorder=3)
        
    plt.title(f'{title_suffix}\nPC1: {exp_var[0]:.1%} var, PC2: {exp_var[1]:.1%} var', fontsize=20)
    plt.xlabel('Principal Component 1', fontsize=18)
    plt.ylabel('Principal Component 2', fontsize=18)
    
    ax.tick_params(axis='both', which='major', labelsize=14)
    
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', frameon=False, title="Stage", 
               title_fontsize=16, fontsize=14)
    
    plt.grid(True, alpha=0.2, linestyle='--')
    plt.tight_layout()
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Generated refined plot: {output_filename}")

# --- Helper Function  ---
def confidence_ellipse(x, y, ax, n_std=2.0, facecolor='none', **kwargs):
    if x.size < 2 or y.size < 2: return None
    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)
    transf = transforms.Affine2D().rotate_deg(45).scale(scale_x, scale_y).translate(mean_x, mean_y)
    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

# --- Execution ---
if not pivot_pct.empty:
    # Plot 1: All Data
    plot_pca_refined_v3(pivot_pct, 
                          "PCA: All Data (Large Fonts & Black Trajectory)", 
                          "/public/home/design1/data/cellranger/data_CARNK_START/output/pca_all_data_final_v3.pdf")
else:
    print("Not enough data")

In [ ]:
import scanpy as sc
import gseapy as gp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from adjustText import adjust_text 

# ------------------------------------------------------------------
# ------------------------------------------------------------------
OUT_DIR = '/public/home/design1/data/cellranger/data_CARNK_START/output/'
os.makedirs(OUT_DIR, exist_ok=True)

# ------------------------------------------------------------------
# 1. Load Data (Ensure it's the merged object with all stages)
# ------------------------------------------------------------------
# adata_B = sc.read('/public/home/design1/data/cellranger/data_CARNK_START/h5ad/adata_B_1128.h5ad')

groupby_col = 'stage' 

adata_subset = adata_B[adata_B.obs[groupby_col].isin(['D14', 'M3'])].copy()

# ------------------------------------------------------------------
# 2.DEGs
# ------------------------------------------------------------------
sc.tl.rank_genes_groups(
    adata_subset, 
    groupby=groupby_col, 
    groups=['M3'],       
    reference='D14',     
    method='wilcoxon',
    pts=True,           
    key_added='rank_genes_reconstitution'
)

dedf = sc.get.rank_genes_groups_df(
    adata_subset, 
    group='M3', 
    key='rank_genes_reconstitution'
).dropna()

# THRESHOLD
logfc_threshold = 1 
padj_threshold = 0.05 

dedf['significance'] = np.where(
    (abs(dedf['logfoldchanges']) > logfc_threshold) & (dedf['pvals_adj'] < padj_threshold),
    'Significant',
    'Not significant'
)

# ------------------------------------------------------------------
# 3. GSEA Prerank 
# ------------------------------------------------------------------
rnk = dedf[['names', 'scores']].sort_values('scores', ascending=False)

pre_res = gp.prerank(
    rnk=rnk, 
    gene_sets='GO_Biological_Process_2023',
    threads=4,
    min_size=10,
    max_size=1000,
    seed=1024
)
pre_res.res2d.index = pre_res.res2d.index.astype(str)

# Save full GSEA results to CSV
all_results_df = pre_res.res2d.copy()

all_results_df.index = all_results_df.index.astype(str)

#Sort by FDR q-value
all_results_df = all_results_df.sort_values('FDR q-val')

full_csv_path = os.path.join(OUT_DIR, 'GSEA_M3_vs_D14_All_Pathways_Full.csv')

all_results_df.to_csv(full_csv_path, index=True)

